<a href="https://colab.research.google.com/github/KeisukeShimokawa/CarND-Advanced-Lane-Lines/blob/master/part1/lesson18_project4_Behavioral_Cloning_More_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson18 Bahavioral Cloning More Data

## SetUp

In [0]:
%tensorflow_version 1.x

In [0]:
!wget -q https://www.dropbox.com/s/802qjuws26wp0ww/data-rev.zip -O data-rev.zip
!unzip -q -o data-rev.zip
!rm data-rev.zip

!wget -q https://www.dropbox.com/s/g0bqpgstzv2ojv7/data.zip -O data.zip
!unzip -q -o data.zip
!rm data.zip

In [3]:
!ls

data  data-rev	sample_data


In [4]:
import tensorflow as tf
import keras

print(tf.__version__)
print(keras.__version__)

1.15.0
2.2.5


Using TensorFlow backend.


## fit_generator

In [0]:
import os
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

In [0]:
def get_lines_from_file(path):
    
    assert os.path.exists(path), "path does not exist!!"
    
    lines = []
    with open(path) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            lines.append(line)
    return lines

In [0]:
def combine_multiple_lines(*args):
    print('we got {} lines'.format(len(args)))

    combine_lines = []
    for arg in args:
        combine_lines.extend(arg)

    return combine_lines

In [0]:
def get_all_lines(*paths):

    lines = []
    for path in paths:
        lines.append(get_lines_from_file(path))

    all_lines = combine_multiple_lines(*lines)
    return all_lines

In [9]:
all_path = [
    'data/driving_log.csv', 
    'data-rev/driving_log.csv'
]
lines = get_all_lines(*all_path)

we got 2 lines


In [10]:
len(lines)

9724

In [0]:
def get_fname(path):
    return '/'.join(path.split('/')[-3:])

In [12]:
get_fname(lines[-1][0])

'data-rev/IMG/center_2020_03_01_12_38_12_234.jpg'

In [13]:
get_fname(lines[0][0])

'data/IMG/center_2020_03_01_12_10_18_792.jpg'

## Generator

In [0]:
train_lines, valid_lines = train_test_split(lines, test_size=0.2, 
                                            random_state=0, shuffle=True)

In [0]:
class BatchGenerator(keras.utils.Sequence):
    def __init__(self, lines, batch_size, target_image_dir='/opt/data/', is_both_side=False, is_flip=True):
        self.lines = lines
        self.target_image_dir = target_image_dir
        self.is_both_side = is_both_side
        self.is_flip = is_flip

        self.batch_size = batch_size
        self.actual_batch_size = batch_size
        if is_both_side: self.actual_batch_size *= 3
        if is_flip: self.actual_batch_size *= 2
        print('actual batch size: {}'.format(self.actual_batch_size))

        self.length = len(lines)
        if is_both_side: self.length *= 3
        if is_flip: self.length *= 2
        print('total data length: {}'.format(self.length))

        self.batches_per_epoch = int((self.length - 1) / self.actual_batch_size) + 1
        print('batches per epoch: {}'.format(self.batches_per_epoch))

    def __getitem__(self, idx):
        batch_from = self.batch_size * idx
        batch_to = batch_from + self.batch_size

        if batch_to > self.length:
            batch_to = self.length

        x_batch, y_batch = self.get_image_and_steering(self.lines[batch_from:batch_to],
                                                      self.target_image_dir,
                                                      self.is_both_side,
                                                      self.is_flip)

        x_batch = np.array(x_batch)
        y_batch = np.array(y_batch)
        return x_batch, y_batch

    def __len__(self):
        return self.batches_per_epoch

    def on_epoch_end(self):
        pass

    def get_image_and_steering(self, lines, target_image_dir='/opt/IMG/', is_both_side=True, is_flip=True):
        n_side = 3 if is_both_side else 1
        images = []
        measurements = []
        
        for line in lines:
            correction = 0.2         
            steering = float(line[3])
            
            for i in range(n_side):    
                if i == 1:
                    # if camera is set on left
                    steering += correction
                elif i == 2:
                    # if camera is set on right
                    steering -= correction
                    
                image_path = target_image_dir + get_fname(line[i])
                assert os.path.exists(image_path), "image path is wrong: {}".format(image_path)
                image = cv2.imread(image_path)
                
                images.append(image)
                measurements.append(steering)
                
                if is_flip:
                    images.append(cv2.flip(image, 1))
                    measurements.append(steering * -1.0)
                
        return images, measurements

In [16]:
train_batch_generator = BatchGenerator(train_lines, 
                                       batch_size=64, 
                                       target_image_dir='./',
                                       is_both_side=False,
                                       is_flip=True)
valid_batch_generator = BatchGenerator(valid_lines, 
                                       batch_size=100, 
                                       target_image_dir='./',
                                       is_both_side=False,
                                       is_flip=False)

actual batch size: 128
total data length: 15558
batches per epoch: 122
actual batch size: 100
total data length: 1945
batches per epoch: 20


In [17]:
len(train_lines), len(valid_lines)

(7779, 1945)

In [18]:
len(train_lines)*6, len(valid_lines)*6

(46674, 11670)

## Model

In [0]:
def get_model(im_size, cropping):
    model = Sequential()
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=im_size))
    model.add(Cropping2D(cropping=cropping))
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(64, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    return model

In [20]:
im_size = (160, 320, 3)
cropping = ((70, 20), (0, 0))

model = get_model(im_size, cropping)
model.compile(loss='mse', optimizer='adam')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [21]:
history = model.fit_generator(
           train_batch_generator, 
           steps_per_epoch=train_batch_generator.batches_per_epoch, 
           validation_data=valid_batch_generator, 
           validation_steps=valid_batch_generator.batches_per_epoch,
           epochs=15,
           shuffle=True,
           verbose=1)




Epoch 1/15





122/122 [==============================] - 33s 269ms/step - loss: 0.0614 - val_loss: 0.0484
Epoch 2/15
122/122 [==============================] - 17s 139ms/step - loss: 0.0478 - val_loss: 0.0479
Epoch 3/15
122/122 [==============================] - 17s 139ms/step - loss: 0.0463 - val_loss: 0.0466
Epoch 4/15
122/122 [==============================] - 17s 138ms/step - loss: 0.0437 - val_loss: 0.0463
Epoch 5/15
122/122 [==============================] - 17s 139ms/step - loss: 0.0434 - val_loss: 0.0391
Epoch 6/15
122/122 [==============================] - 17s 138ms/step - loss: 0.0418 - val_loss: 0.0403
Epoch 7/15
122/122 [==============================] - 17s 138ms/step - loss: 0.0415 - val_loss: 0.0419
Epoch 8/15
122/122 [==============================] - 17s 139ms/step - loss: 0.0409 - val_loss: 0.0452
Epoch 9/15
122/122 [==============================] - 17s 138ms/step - loss: 0.0400 - val_loss: 0.0412
Epoch 10/15
122/122 [==============================] - 17s 139ms/

In [22]:
history.history['loss'], history.history['val_loss']

([0.061413918807767386,
  0.04788120266043875,
  0.04632605899133479,
  0.04375583327265567,
  0.043474990736847824,
  0.0419028003876429,
  0.04156995443180673,
  0.04101105282775869,
  0.040044984604139945,
  0.038630625635082856,
  0.03852243580451844,
  0.03810701731974093,
  0.036852502758896184,
  0.035560514308346516,
  0.03501332010381831],
 [0.04837116311785494,
  0.047922977483364486,
  0.04660340149558908,
  0.04627242421965979,
  0.03911175488812758,
  0.04031124143145385,
  0.04193467914981523,
  0.04517626287391682,
  0.041161064259650465,
  0.04026066779056061,
  0.049338476162566626,
  0.04208116828054573,
  0.03996271428419876,
  0.04014404277989981,
  0.03743145225095565])

In [0]:
model.save('model.h5')